<a href="https://colab.research.google.com/github/froggagul/nsmc/blob/master/Naver_sentiment_movie_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# download raw datasets
import requests


f_train = requests.get('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
f_test = requests.get('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt')

open('train.txt', 'wb').write(f_train.content)
open('test.txt', 'wb').write(f_test.content)

4893335

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.8 MB 4.1 MB/s 
     |████████████████████████████████| 3.3 MB 21.1 MB/s 
     |████████████████████████████████| 895 kB 60.2 MB/s 
     |████████████████████████████████| 636 kB 70.7 MB/s 
     |████████████████████████████████| 52 kB 2.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizerFast, BertModel, AdamW
from tqdm.notebook import tqdm

# reproducible pytorch
torch.manual_seed(1234)

In [ ]:
class NSMCDataset(Dataset):
  
    def __init__(self, file_path):
        self.dataset = pd.read_csv(file_path, sep='\t')
        
        # drop nan row
        self.dataset = self.dataset.dropna(axis = 0)
        # drop duplicate row
        # self.dataset['document'] = self.dataset['document'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)
        self.dataset.drop_duplicates(subset=['document'], inplace=True)
        
        # tokenizer
        self.tokenizer = BertTokenizerFast.from_pretrained("kykim/bert-kor-base")
  
        # add special tokens
        # self.tokenizer.add_tokens(["OO", "OOO", "OOOO"], special_tokens=True)
        
    def __len__(self):
        return len(self.dataset)
  
    def __getitem__(self, idx):
        row = self.dataset.iloc[idx, 1:3].values # no ids!
    
        '''
        ["document", "label"]
        '''
        document = row[0]
        label = row[1]

        inputs = self.tokenizer(
            document, 
            return_tensors='pt',
            truncation=True,
            max_length=256,
            pad_to_max_length=True,
            add_special_tokens=True
            )

        input_ids = inputs['input_ids'][0]
        attention_mask = inputs['attention_mask'][0]

        return input_ids, attention_mask, label

In [ ]:
train_dataset = NSMCDataset("train.txt")
test_dataset = NSMCDataset("test.txt")

Downloading:   0%|          | 0.00/344k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/725 [00:00<?, ?B/s]

In [ ]:
class BertClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert_model = BertModel.from_pretrained("kykim/bert-kor-base")
        self.fc = nn.Linear(768, 2)
    def forward(self, input_ids, attention_mask):
        outputs = self.bert_model(input_ids = input_ids, attention_mask = attention_mask)

        pooled_output = outputs[1]
        return self.fc(pooled_output)

In [ ]:
device = torch.device("cuda")

In [ ]:
model = BertClassifier()

model.to(device)

model.bert_model.resize_token_embeddings(len(train_dataset.tokenizer))

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

Some weights of the model checkpoint at kykim/bert-kor-base were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Embedding(42000, 768, padding_idx=0)

In [ ]:
epochs = 4
batch_size = 16

In [ ]:
optimizer = AdamW(model.parameters(), lr=1e-5)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=True)

In [ ]:
losses = []
accuracies = []

for epoch in range(1, epochs + 1):
    total_loss = 0.0
    correct = 0
    total = 0
    batch_index = 0

    model.train()

    for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
        optimizer.zero_grad()

        y_batch = y_batch.to(device)
        input_ids_batch = input_ids_batch.to(device)
        attention_masks_batch = attention_masks_batch.to(device)
        
        y_pred = model(input_ids=input_ids_batch, attention_mask=attention_masks_batch)
        loss = F.cross_entropy(y_pred, y_batch)

        loss.backward()

        optimizer.step()

        total_loss += loss.item()

        predicted = torch.max(y_pred, 1)[1]
        correct += (predicted == y_batch).sum()
        total += len(y_batch)

        batch_index += 1
        if batch_index % 500 == 0:
            print(f"epoch #{epoch} {batch_index} Batch Loss:{total_loss} Accuracy:{correct.float() / total}")

    losses.append(total_loss)
    accuracies.append(correct.float() / total)
    print("Train Loss:", total_loss, "Accuracy:", correct.float() / total)
    torch.save(model.state_dict(), f"model_{epoch}_base.pt")

  0%|          | 0/9137 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2204: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


epoch #1 500 Batch Loss:191.1491790264845 Accuracy:0.830875039100647
epoch #1 1000 Batch Loss:340.790211699903 Accuracy:0.8542500138282776
epoch #1 1500 Batch Loss:487.37388014420867 Accuracy:0.8612083196640015
epoch #1 2000 Batch Loss:626.4321026466787 Accuracy:0.866812527179718
epoch #1 2500 Batch Loss:770.3852730803192 Accuracy:0.8683750033378601
epoch #1 3000 Batch Loss:904.609647359699 Accuracy:0.8716874718666077
epoch #1 3500 Batch Loss:1031.990282887593 Accuracy:0.8754464387893677
epoch #1 4000 Batch Loss:1162.257105531171 Accuracy:0.8776562809944153
epoch #1 4500 Batch Loss:1289.3788650482893 Accuracy:0.879111111164093
epoch #1 5000 Batch Loss:1415.2494589295238 Accuracy:0.8807999491691589
epoch #1 5500 Batch Loss:1539.7998806778342 Accuracy:0.882011353969574
epoch #1 6000 Batch Loss:1662.06950542517 Accuracy:0.8834478855133057
epoch #1 6500 Batch Loss:1784.9396831989288 Accuracy:0.8846730589866638
epoch #1 7000 Batch Loss:1906.3767006443813 Accuracy:0.8853928446769714
epoch #1

  0%|          | 0/9137 [00:00<?, ?it/s]

epoch #2 500 Batch Loss:94.00427514128387 Accuracy:0.924750030040741
epoch #2 1000 Batch Loss:187.44460872933269 Accuracy:0.9250625371932983
epoch #2 1500 Batch Loss:289.1784286350012 Accuracy:0.9229999780654907
epoch #2 2000 Batch Loss:389.4412902444601 Accuracy:0.9230000376701355
epoch #2 2500 Batch Loss:482.33210276812315 Accuracy:0.9242749810218811


KeyboardInterrupt: ignored

In [ ]:
model.eval()

test_correct = 0
test_total = 0

for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))
    predicted = torch.max(y_pred, 1)[1]
    test_correct += (predicted == y_batch).sum()
    test_total += len(y_batch)

print("Accuracy:", test_correct.float() / test_total)

  0%|          | 0/1537 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2204: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


RuntimeError: ignored

In [ ]:
losses, accuracies

In [ ]:
!nvidia-smi

Mon Sep 20 04:01:20 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive
